<a href="https://colab.research.google.com/github/Nusserle/LieDetection/blob/master/LieDetectionEEG_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
import scipy
from scipy import signal

plt.rcParams['figure.figsize'] = (18, 6)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/My Drive/Shared/BagOfLies/
df = pd.read_csv("Annotations.csv")
df_eeg = df[['eeg', 'truth']].dropna().reset_index()
df_eeg.head()

/content/drive/My Drive/Shared/BagOfLies


,index,eeg,truth
0,0,./Finalised/User_0/run_0/EEG.csv,1
1,1,./Finalised/User_0/run_1/EEG.csv,0
2,2,./Finalised/User_0/run_2/EEG.csv,0
3,3,./Finalised/User_0/run_3/EEG.csv,0
4,4,./Finalised/User_0/run_4/EEG.csv,0


In [0]:
# Filter
filter = signal.firwin(600, [0.01, 0.06], pass_zero=False)

In [5]:
length_of_trial = []
data = []
label = []
for i, eeg in df_eeg.iterrows():
  if i % 20 == 0:
    print("Processed trials:", i)
  df_trial = pd.read_csv(eeg[1]).iloc[:,1:-1]
  # Convert to numeric and drop N/A values
  df_trial = df_trial.apply(lambda x: pd.to_numeric(x, errors = 'coerce')).dropna()
  # Filter EEG table only for values (except for X and Y Value)
  df_trial = df_trial.loc[:, df_trial.columns.str.contains('Value')]
  if i >= 1:
    length_of_trial.append((length_of_trial[i-1][1]+1, length_of_trial[i-1][1] + df_trial.shape[0]))
  else:
    length_of_trial.append((0, df_trial.shape[0]))
  for col in df_trial:
    # Normalize and filter
    data_norm_to_0_1 = [number/scipy.linalg.norm(df_trial[col].squeeze()) for number in df_trial[col].squeeze()]
    filtered_and_normalized = signal.convolve(data_norm_to_0_1, filter, mode="same")
    df_trial[col] = filtered_and_normalized
    df_trial["label"] = eeg[2]
  for j, row in df_trial.iterrows():
    data.append(row.values[:-1])
    label.append(row.values[-1])

  #for j, el in enumerate(combined_vec):
  #  data[i][j] = combined_vec[j]
  
  #min_max_scaler = preprocessing.MinMaxScaler()
  #x_scaled = min_max_scaler.fit_transform(x)
  
  #label.append(eeg[2])
#label = np.asarray(label)
print(data.__len__())
print(label.__len__())
data = np.asarray(data)
label = np.asarray(label)

Processed trials: 0
Processed trials: 20
Processed trials: 40
Processed trials: 60
Processed trials: 80
Processed trials: 100
Processed trials: 120
Processed trials: 140
Processed trials: 160
Processed trials: 180
Processed trials: 200
271427
271427


In [6]:
accuracies = []
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(length_of_trial, length_of_trial):
  X_train, y_train = [], []
  for index in train_index:
    start, end = length_of_trial[index]
    for row in data[start:end]:
      X_train.append(row)
    for row in label[start:end]:
      y_train.append(row)
  learned_model = GaussianNB().fit(X_train, y_train)

  score = 0
  for index in test_index:
    X_test, y_test = [], []
    start, end = length_of_trial[index]
    for row in data[start:end]:
      X_test.append(row)
    for row in label[start:end]:
      y_test.append(row)
    prediction = learned_model.predict_proba(np.asarray(X_test))
    mysum = 0
    for p in prediction:
      mysum += p[1]
    if mysum/X_test.__len__() >= 0.5:
    #if sum(prediction == 1) >= X_test.__len__() / 2:
      pred = 1
    else:
      pred = 0
    if y_test[0] == pred:
      score += 1
  accuracies.append(score/test_index.__len__())
  print(score/test_index.__len__())
print("Model {} with avg test accuracy: {}".format(GaussianNB(), sum(accuracies)/accuracies.__len__()))  

0.6829268292682927
0.5
0.625
0.525
0.55
Model GaussianNB(priors=None, var_smoothing=1e-09) with avg test accuracy: 0.5765853658536585
